In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df1= pd.read_csv(r"C:\Users\i\Downloads\joy-ratings-0to1.train.txt", delimiter='\t', header=None)
df1.columns = ['Id', 'tweet', 'emotion', 'score']

In [3]:
print(df1)


        Id                                              tweet emotion  score
0    30000  Just got back from seeing @GaryDelaney in Burs...     joy  0.980
1    30001  Oh dear an evening of absolute hilarity I don'...     joy  0.958
2    30002  Been waiting all week for this game ❤️❤️❤️ #ch...     joy  0.940
3    30003  @gardiner_love : Thank you so much, Gloria! Yo...     joy  0.938
4    30004  I feel so blessed to work with the family that...     joy  0.938
..     ...                                                ...     ...    ...
818  30818  It's just the lack of company and liveliness o...     joy  0.058
819  30819             Quinn's short hair makes me sad. #glee     joy  0.040
820  30820  hate overthinking e v e r y t h i n g like i j...     joy  0.040
821  30821  People who cheer for sports teams completely o...     joy  0.020
822  30822  @DamnPatriot You're a POS for rejoicing in som...     joy  0.019

[823 rows x 4 columns]


In [4]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import emoji
import re

In [5]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df1['tweet'] = df1['tweet'].apply(preprocess_tweet)

In [6]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df1['tweet'] = df1['tweet'].apply(remove_mentions)


In [7]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text) 
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df1['tweet'] = df1['tweet'].apply(clean_text)



In [8]:
df1

,Id,tweet,emotion,score
0,30000,just got back from seeing in burslem amazing f...,joy,0.980
1,30001,oh dear an evening of absolute hilarity i dont...,joy,0.958
2,30002,been waiting all week for this game #cheer #fr...,joy,0.940
3,30003,thank you so much gloria youre so sweet and th...,joy,0.938
4,30004,i feel so blessed to work with the family that...,joy,0.938
...,...,...,...,...
818,30818,its just the lack of company and liveliness ou...,joy,0.058
819,30819,quinns short hair makes me sad #glee,joy,0.040
820,30820,hate overthinking e v e r y t h i n g like i j...,joy,0.040
821,30821,people who cheer for sports teams completely o...,joy,0.020


In [9]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text

In [10]:
df1['tweet'] = df1['tweet'].apply(convert_emoji)

In [11]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df1['tweet'] = df1['tweet'].apply(tokenize_tweets)

In [12]:
df1

,Id,tweet,emotion,score
0,30000,"[just, got, back, from, seeing, in, burslem, a...",joy,0.980
1,30001,"[oh, dear, an, evening, of, absolute, hilarity...",joy,0.958
2,30002,"[been, waiting, all, week, for, this, game, #c...",joy,0.940
3,30003,"[thank, you, so, much, gloria, youre, so, swee...",joy,0.938
4,30004,"[i, feel, so, blessed, to, work, with, the, fa...",joy,0.938
...,...,...,...,...
818,30818,"[its, just, the, lack, of, company, and, livel...",joy,0.058
819,30819,"[quinns, short, hair, makes, me, sad, #glee]",joy,0.040
820,30820,"[hate, overthinking, e, v, e, r, y, t, h, i, n...",joy,0.040
821,30821,"[people, who, cheer, for, sports, teams, compl...",joy,0.020


In [13]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'all',
    'about', 'above', 'after', 'again', 'against', 'ain', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
    'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can',
    'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't",
    'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't",
    'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how',
    'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma',
    'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no',
    'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves',
    'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn',
    "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them',
    'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until',
    'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where',
    'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you',
    "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens

df1['tweet'] = df1['tweet'].apply(remove_stopwords)

In [14]:
df1

,Id,tweet,emotion,score
0,30000,"[got, back, seeing, burslem, amazing, face, st...",joy,0.980
1,30001,"[oh, dear, evening, absolute, hilarity, dont, ...",joy,0.958
2,30002,"[waiting, week, game, #cheer, #friday]",joy,0.940
3,30003,"[thank, much, gloria, youre, sweet, thoughtful...",joy,0.938
4,30004,"[feel, blessed, work, family, nanny, nothing, ...",joy,0.938
...,...,...,...,...
818,30818,"[lack, company, liveliness, makes, bored]",joy,0.058
819,30819,"[quinns, short, hair, makes, sad, #glee]",joy,0.040
820,30820,"[hate, overthinking, like, jus, wanna, happy, ...",joy,0.040
821,30821,"[people, cheer, sports, teams, completely, out...",joy,0.020


In [15]:
import torch
from transformers import BertModel, BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def extract_features(tweet):
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    
    return features

df1['features'] = df1['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
from sklearn.utils import shuffle
df1 = shuffle(df1)
df1

,Id,tweet,emotion,score,features
217,30217,"[yeah, playing, well, #optimism]",joy,0.625,"[[-0.09087066, 0.20415401, 0.38680488, 0.03166..."
769,30769,"[would, like, know, source, presidents, optimi...",joy,0.180,"[[-0.10396489, -0.044955045, -0.051590685, 0.0..."
371,30371,"[man, know, juggle, didnt, balls, #funny, #pun...",joy,0.519,"[[-0.10058107, 0.23081657, -0.1354249, -0.0073..."
711,30711,"[god, facebooks, design, started, remind, mysp...",joy,0.250,"[[0.13354683, -0.04225903, -0.053082053, -0.25..."
414,30414,"[hes, brilliant, lost, joyous, plot, us, year,...",joy,0.480,"[[-0.4616325, -0.026620476, 0.21807232, 0.0884..."
...,...,...,...,...,...
812,30812,"[quinns, short, hair, makes, sad]",joy,0.083,"[[-0.011215197, 0.08818388, -0.063707456, 0.01..."
340,30340,"[watched, django, unchained, people, may, frow...",joy,0.536,"[[0.15952902, 0.13285297, 0.2045169, 0.0801301..."
449,30449,"[watch, amazing, lively, broadcast, #musically]",joy,0.462,"[[-0.027367242, -0.017335942, 0.23272076, -0.0..."
594,30594,"[yell, bye, garbage, cheerfully]",joy,0.360,"[[0.1024229, 0.5135636, -0.030556034, -0.06082..."


In [17]:
df1.features[0].shape

(16, 768)

In [18]:

df1.head(10)

,Id,tweet,emotion,score,features
217,30217,"[yeah, playing, well, #optimism]",joy,0.625,"[[-0.09087066, 0.20415401, 0.38680488, 0.03166..."
769,30769,"[would, like, know, source, presidents, optimi...",joy,0.180,"[[-0.10396489, -0.044955045, -0.051590685, 0.0..."
371,30371,"[man, know, juggle, didnt, balls, #funny, #pun...",joy,0.519,"[[-0.10058107, 0.23081657, -0.1354249, -0.0073..."
711,30711,"[god, facebooks, design, started, remind, mysp...",joy,0.250,"[[0.13354683, -0.04225903, -0.053082053, -0.25..."
414,30414,"[hes, brilliant, lost, joyous, plot, us, year,...",joy,0.480,"[[-0.4616325, -0.026620476, 0.21807232, 0.0884..."
770,30770,"[unixware, skydrivecokie, hearty, exactment, b...",joy,0.180,"[[-0.24502039, 0.23167601, -0.18355255, -0.050..."
232,30232,"[god, replaced, sadness, laughter, cant, go, w...",joy,0.615,"[[-0.14044082, 0.38514432, 0.18663892, -0.0345..."
779,30779,"[apparently, money, world, cant, buy, single, ...",joy,0.160,"[[-0.19860527, 0.02035996, -0.018056665, 0.059..."
527,30527,"[intermit, animatedncaa, pittsburgh, vs, missi...",joy,0.406,"[[-0.64525056, -0.26255015, -0.029385332, -0.2..."
223,30223,"[man, know, juggle, didnt, balls, itn, #funny,...",joy,0.625,"[[-0.004961823, 0.425498, -0.117140874, 0.0105..."


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features = df1['features'].tolist()
padded_features = pad_sequences(features, padding='post')
padded_df = df1.copy()
padded_df['features'] = padded_features.tolist()

In [20]:
x = np.stack(padded_df['features'])

In [21]:
y = np.array(padded_df['score'])    
print("Input shape:", x.shape)
print("Output shape:", y.shape)

Input shape: (823, 41, 768)
Output shape: (823,)


In [22]:
x = np.reshape(x, (823, 41 * 768))  
y = np.reshape(y, (823,))

In [23]:
y

array([0.625, 0.18 , 0.519, 0.25 , 0.48 , 0.18 , 0.615, 0.16 , 0.406,
       0.625, 0.438, 0.417, 0.562, 0.24 , 0.483, 0.407, 0.104, 0.75 ,
       0.958, 0.48 , 0.179, 0.38 , 0.66 , 0.229, 0.458, 0.333, 0.3  ,
       0.34 , 0.173, 0.28 , 0.417, 0.312, 0.479, 0.74 , 0.04 , 0.292,
       0.44 , 0.3  , 0.864, 0.395, 0.815, 0.46 , 0.667, 0.708, 0.438,
       0.5  , 0.6  , 0.396, 0.36 , 0.312, 0.562, 0.438, 0.708, 0.48 ,
       0.688, 0.64 , 0.542, 0.729, 0.288, 0.32 , 0.4  , 0.237, 0.602,
       0.458, 0.06 , 0.327, 0.479, 0.75 , 0.56 , 0.353, 0.46 , 0.56 ,
       0.312, 0.833, 0.646, 0.481, 0.462, 0.578, 0.558, 0.312, 0.833,
       0.86 , 0.48 , 0.245, 0.667, 0.708, 0.625, 0.562, 0.24 , 0.68 ,
       0.36 , 0.94 , 0.84 , 0.1  , 0.417, 0.542, 0.68 , 0.812, 0.6  ,
       0.68 , 0.34 , 0.562, 0.5  , 0.771, 0.52 , 0.229, 0.28 , 0.729,
       0.167, 0.46 , 0.611, 0.72 , 0.646, 0.66 , 0.78 , 0.312, 0.18 ,
       0.521, 0.926, 0.292, 0.562, 0.78 , 0.521, 0.25 , 0.14 , 0.558,
       0.48 , 0.625,

In [24]:
X=x.copy()  
Y=y.copy()  

In [25]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

alpha = 2000
model = Ridge(alpha=alpha)
model.fit(X, Y)

Ridge(alpha=1500)

In [26]:
y_pred = model.predict(X)
mse = mean_squared_error(Y, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 0.02252365870889028


In [27]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y, y_pred)

print("Mean Absolute Error:", mae)

Mean Absolute Error: 0.1218327461509717


In [28]:
import joblib
save_path = 'regression_model.pkl'
joblib.dump(model, save_path)


['regression_model.pkl']

In [29]:
df2= pd.read_csv(r"C:\Users\i\Downloads\joy-ratings-0to1.dev.gold.txt", delimiter='\t', header=None)
df2.columns = ['Id', 'tweet', 'emotion', 'score']

In [30]:
df2

,Id,tweet,emotion,score
0,30823,"@theclobra lol I thought maybe, couldn't decid...",joy,0.312
1,30824,Nawaz Sharif is getting more funnier than @kap...,joy,0.700
2,30825,Nawaz Sharif is getting more funnier than @kap...,joy,0.580
3,30826,@tomderivan73 😁...I'll just people watch and e...,joy,0.438
4,30827,I love my family so much #lucky #grateful #sma...,joy,0.936
...,...,...,...,...
74,30897,It feels good to get outside for a minute and ...,joy,0.580
75,30898,@r0Ils ppl get triggered over u smiling they'r...,joy,0.170
76,30899,@GigaFag @pipertownsend_ snapchat new would be...,joy,0.396
77,30900,@GigaFag @pipertownsend_ snapchat new would be...,joy,0.156


In [31]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df2['tweet'] = df2['tweet'].apply(preprocess_tweet)

In [32]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df2['tweet'] = df2['tweet'].apply(remove_mentions)


In [33]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text)  
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df2['tweet'] = df2['tweet'].apply(clean_text)


In [34]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text
df2['tweet'] = df2['tweet'].apply(convert_emoji)

In [35]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df2['tweet'] = df2['tweet'].apply(tokenize_tweets)

In [36]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'all',
    'about', 'above', 'after', 'again', 'against', 'ain', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
    'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can',
    'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't",
    'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't",
    'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how',
    'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma',
    'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no',
    'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves',
    'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn',
    "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them',
    'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until',
    'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where',
    'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you',
    "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']


def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens
df2['tweet'] = df2['tweet'].apply(remove_stopwords)

In [37]:
import torch
from transformers import BertModel, BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


def extract_features(tweet):
    
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    
    return features
df2['features'] = df2['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features2 = df2['features'].tolist()
padded_features2 = pad_sequences(features2, padding='post')
padded_df2 = df2.copy()
padded_df2['features'] = padded_features2.tolist()

In [39]:
X = np.stack(padded_df2['features'])
print('Input feature shape:', X.shape)

Input feature shape: (79, 30, 768)


In [40]:
print('Input feature shape:', X.shape)

Input feature shape: (79, 30, 768)


In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length=41
X_=pad_sequences(X,maxlen=max_sequence_length,padding="post",truncating='post')

In [42]:
X_ = np.reshape(X_, (79, 41 * 768))  

In [43]:
saved_model_path = 'regression_model.pkl'
loaded_model = joblib.load(saved_model_path)

In [44]:
predictions = loaded_model.predict(X_)
predictions

array([0.51067087, 0.56594957, 0.53886009, 0.42587583, 0.67197353,
       0.62279419, 0.42493034, 0.38395827, 0.43890411, 0.46397063,
       0.44422341, 0.44984654, 0.49561138, 0.51144366, 0.54812439,
       0.41797186, 0.5506133 , 0.51982122, 0.50336815, 0.49420508,
       0.49448108, 0.50812564, 0.43621751, 0.50508995, 0.46945534,
       0.49814222, 0.51606673, 0.43800663, 0.4931089 , 0.5102105 ,
       0.56497422, 0.53859552, 0.57704407, 0.56196774, 0.44054848,
       0.51594501, 0.38864627, 0.42835729, 0.42044819, 0.5535203 ,
       0.46597232, 0.43993218, 0.54864473, 0.72180125, 0.49060816,
       0.64108801, 0.47002779, 0.5060718 , 0.50455816, 0.4811201 ,
       0.45205043, 0.46731664, 0.44736971, 0.45089417, 0.41492245,
       0.40951056, 0.51635703, 0.44873437, 0.36714014, 0.47626151,
       0.50994082, 0.44523467, 0.46325495, 0.47538883, 0.46217677,
       0.46498087, 0.43780302, 0.44721916, 0.47382236, 0.48220281,
       0.51320674, 0.49671024, 0.45515152, 0.43341137, 0.47332

In [45]:
df2['predict']=pd.DataFrame(predictions)

In [46]:
mse=mean_squared_error(df2['score'],df2['predict'])
print("mean_square_error:",mse)

mean_square_error: 0.0359444478046534


In [47]:
mae = mean_absolute_error(df2['score'],df2['predict'])
print("Mean Absolute Error:", mae)


Mean Absolute Error: 0.16298924407690876


In [48]:
df2

,Id,tweet,emotion,score,features,predict
0,30823,"[lol, thought, maybe, couldnt, decide, levity]",joy,0.312,"[[-0.26824236, 0.33785567, -0.06891699, 0.1938...",0.510671
1,30824,"[nawaz, sharif, getting, funnier, day, day, #l...",joy,0.700,"[[-0.7382417, -0.14251125, 0.09192327, -0.2908...",0.565950
2,30825,"[nawaz, sharif, getting, funnier, day, day, #c...",joy,0.580,"[[-0.77927727, -0.30715537, 0.123740554, -0.08...",0.538860
3,30826,"[ill, people, watch, enjoy, rare, show, optimism]",joy,0.438,"[[0.06172254, 0.24399748, 0.23058335, 0.041538...",0.425876
4,30827,"[love, family, much, #lucky, #grateful, #smart...",joy,0.936,"[[-0.115673274, 0.18784252, 0.052415516, 0.005...",0.671974
...,...,...,...,...,...,...
74,30897,"[feels, good, get, outside, minute, get, fresh...",joy,0.580,"[[0.23290278, 0.111647494, 0.13098769, -0.2015...",0.473327
75,30898,"[ppl, get, triggered, smiling, theyre, irrelev...",joy,0.170,"[[-0.34056798, 0.09875966, 0.037416108, -0.053...",0.423429
76,30899,"[snapchat, new, would, beg, differ, #optimism]",joy,0.396,"[[-0.0088849235, -0.0068749096, -0.09531322, 0...",0.527804
77,30900,"[snapchat, new, would, beg, differ]",joy,0.156,"[[-0.0039193854, -0.053918257, -0.09985137, 0....",0.421061


In [89]:
df3= pd.read_csv(r"C:\Users\i\Downloads\joy-ratings-0to1.test.target.txt", delimiter='\t', header=None)
df3.columns = ['Id', 'tweet', 'emotion', 'score']

In [90]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df3['tweet'] = df3['tweet'].apply(preprocess_tweet)

In [91]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df3['tweet'] = df3['tweet'].apply(remove_mentions)

In [92]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text)  
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df3['tweet'] = df3['tweet'].apply(clean_text)


In [93]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text
df3['tweet'] = df3['tweet'].apply(convert_emoji)

In [94]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df3['tweet'] = df3['tweet'].apply(tokenize_tweets)

In [95]:
stop_words = [
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
    'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with', 'i', 'you', 'your', 'so', 'all',
    'about', 'above', 'after', 'again', 'against', 'ain', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't",
    'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can',
    'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't",
    'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't",
    'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how',
    'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma',
    'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no',
    'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves',
    'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn',
    "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them',
    'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until',
    'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where',
    'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you',
    "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']


def remove_stopwords(tokens):
    filtered_tokens = [token for token in tokens if len(token) > 1 and token.lower() not in stop_words]
    return filtered_tokens
df3['tweet'] = df3['tweet'].apply(remove_stopwords)

In [96]:
import torch
from transformers import BertModel, BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


def extract_features(tweet):
    
    tokenized_text = ' '.join(tweet)
    input_ids = torch.tensor(tokenizer.encode(tokenized_text, add_special_tokens=True)).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input_ids)
    last_hidden_states = outputs.last_hidden_state
    features = last_hidden_states.squeeze(0).numpy()
    
    return features
df3['features'] = df3['tweet'].apply(extract_features)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [97]:
df3['features'][713].shape

(23, 768)

In [98]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

features3 = df3['features'].tolist()
padded_features3 = pad_sequences(features3, padding='post')
padded_df3 = df3.copy()
padded_df3['features'] = padded_features3.tolist()

In [99]:
X = np.stack(padded_df3['features'])
print('Input feature shape:', X.shape)

Input feature shape: (714, 50, 768)


In [100]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length=41
X_=pad_sequences(X,maxlen=max_sequence_length,padding="post",truncating='post')

In [101]:
X_ = np.reshape(X_, (714, 41 * 768))  

In [102]:
saved_model_path = 'regression_model.pkl'
loaded_model = joblib.load(saved_model_path)

In [103]:
predicttest = loaded_model.predict(X_)
predicttest

array([0.49188611, 0.48464327, 0.51110206, 0.48194501, 0.49575611,
       0.49194217, 0.42417449, 0.59917032, 0.60081146, 0.51608042,
       0.46363387, 0.46636145, 0.57816595, 0.5052366 , 0.50554963,
       0.45725766, 0.4754244 , 0.57698626, 0.52600815, 0.45380398,
       0.44849393, 0.53103794, 0.48943426, 0.58125414, 0.48446023,
       0.48433892, 0.48679562, 0.45633936, 0.47808832, 0.4382326 ,
       0.46888897, 0.465714  , 0.4959723 , 0.49959235, 0.52799674,
       0.46330732, 0.42185666, 0.46310266, 0.54037418, 0.44102055,
       0.50585491, 0.46329422, 0.44017728, 0.47596729, 0.42232573,
       0.51357054, 0.52901234, 0.44842178, 0.49648592, 0.45034227,
       0.50658957, 0.57603754, 0.59534921, 0.54607246, 0.54132354,
       0.44128895, 0.5774718 , 0.5050549 , 0.46108631, 0.44969707,
       0.45743111, 0.63194011, 0.54401419, 0.4504699 , 0.43032981,
       0.4900119 , 0.45572501, 0.52789591, 0.51433159, 0.46233172,
       0.48925761, 0.45152229, 0.55285052, 0.54628877, 0.41455

In [104]:
df3['predicttest']=pd.DataFrame(predicttest)

In [105]:
df3=df3.drop(['score','features'],axis=1)

In [106]:
df3

,Id,tweet,emotion,predicttest
0,30902,"[must, knowing, #blithe, means, adj, happy, ch...",joy,0.491886
1,30903,"[old, saying, #smile, shared, one, gained, ano...",joy,0.484643
2,30904,"[bridget, jones, baby, bloody, hilarious, #bri...",joy,0.511102
3,30905,"[sparkling, water, makes, life, sparkly]",joy,0.481945
4,30906,"[im, tired, everybody, telling, chill, everyth...",joy,0.495756
...,...,...,...,...
709,31611,"[tired, body, mind, sparkling, teeth, say, fol...",joy,0.521515
710,31612,"[refuse, chirp, chirp, girl]",joy,0.499702
711,31613,"[hard, stifle, laughter, overheard, comment, r...",joy,0.445443
712,31614,"[walking, little, boy, red, shirt, years, age,...",joy,0.482456
